# eICU Collaborative Research Database

# Workshop 2: Exploring a single patient stay

In [ ]:
# Import libraries
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sbn

In [ ]:
# Plot settings
%matplotlib inline
plt.style.use('ggplot')
fontsize = 20 # size for x and y ticks
plt.rcParams['legend.fontsize'] = fontsize
plt.rcParams.update({'font.size': fontsize})

In [ ]:
# Database config
sqluser = 'postgres'
dbname = 'eicu'
schema_name = 'eicu_crd'

In [ ]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

## 1. Display list of tables

In [ ]:
query = \
"""
SELECT tablename 
FROM pg_catalog.pg_tables 
WHERE schemaname like 'eicu_crd'
ORDER BY tablename;
"""

list_of_tables = pd.read_sql_query(query,con)

In [ ]:
list_of_tables

## 2. Selecting a single patient stay 

### Questions

Using your knowledge from the previous notebook and the online documentation (http://eicu-crd.mit.edu/):

- Which column in the `patient` table is distinct for each stay in the ICU (similar to `icustay_id` in MIMIC-III)?
- Which column is unique for each patient (similar to `subject_id` in MIMIC-III)?


In [ ]:
# select a single ICU stay
patientunitstayid = 141168

In [ ]:
# query to load data from the patient table
query = \
"""
SELECT *
FROM patient
WHERE patientunitstayid = {}
""".format(patientunitstayid)

print(query)

In [ ]:
# run the query and assign the output to a variable
unitstay = pd.read_sql_query(query,con)

In [ ]:
# display the first few rows of the dataframe
unitstay.head()

In [ ]:
# display a list of columns
unitstay.columns

In [ ]:
# select a limited number of columns to view
columns = ['patientunitstayid','gender','age','unitdischargestatus']
unitstay[columns]

In [ ]:
unitstay.describe()

### Questions

- What year was the patient admitted to the ICU? What year was he or she discharged?
- What was the status of the patient upon discharge from the unit?

### 2.2. Periodic vital signs
 
The `vitalperiodic` table comprises data that is consistently interfaced from bedside vital signs monitors into eCareManager. Data are generally interfaced as 1 minute averages, and archived into the `vitalperiodic` table as 5 minute median values. For more detail, see: http://eicu-crd.mit.edu/eicutables/vitalPeriodic/

In [ ]:
# query to load data from the patient table
query = \
"""
SELECT *
FROM vitalperiodic
WHERE patientunitstayid = {}
""".format(patientunitstayid)

print(query)

In [ ]:
# run the query and assign the output to a variable
vitalperiodic = pd.read_sql_query(query,con)

In [ ]:
# display the first few rows of the dataframe
vitalperiodic.head()

In [ ]:
# display a full list of columns
vitalperiodic.columns

In [ ]:
# sort the values by the observationoffset (time in minutes from ICU admission)
vitalperiodic = vitalperiodic.sort_values(by='observationoffset')
vitalperiodic.head()

In [ ]:
# subselect the variable columns
columns = ['observationoffset','temperature','sao2','heartrate','respiration',
          'cvp','etco2','systemicsystolic','systemicdiastolic','systemicmean',
          'pasystolic','padiastolic','pamean','st1','st2','st3','icp']

vitalperiodic = vitalperiodic[columns].set_index('observationoffset')
vitalperiodic.head()

In [ ]:
# plot the data
figsize = (18,8)
title = 'Vital signs from admission to ICU for patientunitstayid = {} \n'.format(patientunitstayid)
ax = vitalperiodic.plot(title=title, figsize=figsize, fontsize=fontsize)

ax.title.set_size(fontsize)
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
ax.set_xlabel("Minutes after admission to the ICU")
ax.set_ylabel("Absolute value")

### Questions

- ???

### 2.3. Aperiodic vital signs¶

In [ ]:
# query to load data from the patient table
query = \
"""
SELECT *
FROM vitalaperiodic
WHERE patientunitstayid = {}
""".format(patientunitstayid)

print(query)

In [ ]:
# run the query and assign the output to a variable
vitalaperiodic = pd.read_sql_query(query,con)

In [ ]:
# display the first few rows of the dataframe
vitalaperiodic.head()

In [ ]:
vitalaperiodic.columns

In [ ]:
# sort the values by the observationoffset (time in minutes from ICU admission)
vitalaperiodic = vitalaperiodic.sort_values(by='observationoffset')
vitalaperiodic.head()

In [ ]:
# subselect the variable columns
columns = ['observationoffset','noninvasivesystolic','noninvasivediastolic',
          'noninvasivemean','paop','cardiacoutput','cardiacinput','svr',
          'svri','pvr','pvri']

vitalaperiodic = vitalaperiodic[columns].set_index('observationoffset')
vitalaperiodic.head()

In [ ]:
# plot the data
figsize = (18,8)
title = 'Vital signs (aperiodic) from admission to ICU for patientunitstayid = {} \n'.format(patientunitstayid)
ax = vitalaperiodic.plot(title=title, figsize=figsize, fontsize=fontsize)

ax.title.set_size(fontsize)
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
ax.set_xlabel("Minutes after admission to the ICU")
ax.set_ylabel("Absolute value")

### 2.4. Infusion drugs

In [ ]:
# query to load data from the patient table
query = \
"""
SELECT *
FROM infusiondrug
WHERE patientunitstayid = {}
""".format(patientunitstayid)

print(query)

In [ ]:
# run the query and assign the output to a variable
infusiondrug = pd.read_sql_query(query,con)

In [ ]:
# display the first few rows of the dataframe
infusiondrug.head()

### 2.5. Laboratory test results

In [ ]:
# query to load data from the patient table
query = \
"""
SELECT *
FROM lab
WHERE patientunitstayid = {}
""".format(patientunitstayid)

print(query)

In [ ]:
# run the query and assign the output to a variable
lab = pd.read_sql_query(query,con)

In [ ]:
# display the first few rows of the dataframe
lab.head()